# Import

In [1]:
import numpy as np
import pandas as pd
import timm
import random
import os
from PIL import Image
import torch

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
import tqdm
from torch import nn
from torch import optim

# Set seed (Reproduce result)

In [2]:
SEED = 123456789

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Split data

In [3]:
data = pd.read_csv('cassava-leaf-disease-classification/train.csv')
choosen_prob = np.random.rand(len(data))
train_df = data[choosen_prob >= 0.2]
val_df = data[choosen_prob < 0.2]

In [4]:
len(val_df)

4294

In [5]:
len(train_df)

17103

# Hyper params

In [6]:
MODEL_NAME = 'tf_efficientnet_b3_ns'
IM_SIZE = 300
BATCH_SIZE = 36
LEARNING_RATE = 1e-3
EPOCH = 50
NUM_WORKER = 8
WEIGHT_DECAY = 1e-4

# Data augmentation

In [7]:
train_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.RandomResizedCrop((IM_SIZE, IM_SIZE)),
     transforms.RandomRotation(90),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

val_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((IM_SIZE, IM_SIZE)),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

# Dataset

In [8]:
class CassavaDataset(Dataset):
    def __init__(self, image_dir, df, transform=None):
        self.image_dir = image_dir
        self.df = df.reset_index(drop=True)
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index]
        label = row.label
        image_name = row.image_id
        
        image = Image.open(os.path.join(self.image_dir, image_name))
        image = np.array(image)
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

In [9]:
train_image_dir = 'cassava-leaf-disease-classification/train_images'
train_dataset = CassavaDataset(train_image_dir, train_df, transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True, 
                                           num_workers=NUM_WORKER)

val_image_dir = 'cassava-leaf-disease-classification/train_images'
val_dataset = CassavaDataset(val_image_dir, val_df, transform=val_transform)
val_loader = torch.utils.data.DataLoader(val_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=False, 
                                           num_workers=NUM_WORKER)

# Model

In [10]:
device = torch.device("cuda")
model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=5)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

model.cuda()
print('Convert model to CUDA')

# for param in model.parameters():
#     param.requires_grad = False

# model.classifier.train()
# model.conv_head.train()
# model.bn2.train()

Convert model to CUDA


# Train and Val

In [12]:
def train_one_epoch(model, train_loader, criterion, optimizer):
    running_loss = 0.0
    scaler = torch.cuda.amp.GradScaler()
    for i, data in enumerate(tqdm.tqdm(train_loader), 0):
        inputs, labels = data
        with torch.cuda.amp.autocast():
            inputs = inputs.cuda()
            labels = labels.cuda()

            outputs = model(inputs)
            assert outputs.dtype is torch.float16
            loss = criterion(outputs, labels)
            assert loss.dtype is torch.float32
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        running_loss += loss.item()
        
    return running_loss / len(train_loader)

In [13]:
def validate(model, val_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm.tqdm(val_loader):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
        
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# def frezze_top(Model):
#     pass

In [ ]:
train_loss_list = []
val_acc_list = []
for epoch in range(EPOCH):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
    val_acc = validate(model, val_loader)
    print("Accuracy at epoch {} is: {}".format(epoch, val_acc))
    train_loss_list.append(train_loss)
    val_acc_list.append(val_acc)

 28%|██▊       | 135/476 [02:17<05:53,  1.04s/it]

# Visualize train loss and val loss

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

print(train_loss_list)
print(val_acc_list)

plt.plot(train_loss_list)
plt.plot(val_acc_list)

plt.xticks(np.arange(0, EPOCH + 1, 1))
plt.yticks(np.arange(0, 100, 0.05))
plt.show()